In [1]:
from neural_network import NeuralNetwork
from neural_network_utility import evaluate
from math_functions.function_enums import LossFunction, ActivationFunction, Metrics
from dataset_reader import read_monk, read_cup, read_old_cup
from validation import kfold_cv, holdout, kfold_cv_ensemble
from grid import grid_search, get_top_n_results, get_all_results
from sklearn.utils import shuffle
from utils import count_configs, save_top_plots, get_list_models, plot_over_epochs
from ensemble import Ensemble
import numpy as np
import json

# Hyperparameter tuning

## Save training data and internal test data

In [ ]:
# data, targets = read_cup('datasets/cup/CUP_TR.csv')
# train_data, test_data, train_targets, test_targets = holdout(data, targets, 0.8, shuffle_set=True)

# np.savetxt(MODEL_SEL_DATA_PATH, train_data, delimiter=',')
# np.savetxt(MODEL_SEL_TARGETS_PATH, train_targets, delimiter=',')
# np.savetxt(MODEL_ASSESS_DATA_PATH, test_data, delimiter=',')
# np.savetxt(MODEL_ASSESS_TARGETS_PATH, test_targets, delimiter=',')

## Load training data and internal test data

In [2]:
MODEL_SEL_DATA_PATH = 'datasets/cup/grid_search/CUP_model_selection_data.csv'
MODEL_SEL_TARGETS_PATH = 'datasets/cup/grid_search/CUP_model_selection_targets.csv'
MODEL_ASSESS_DATA_PATH = 'datasets/cup/grid_search/CUP_model_assessment_data.csv'
MODEL_ASSESS_TARGETS_PATH = 'datasets/cup/grid_search/CUP_model_assessment_targets.csv'

In [3]:
train_data = np.loadtxt(MODEL_SEL_DATA_PATH, delimiter=',')
train_targets = np.loadtxt(MODEL_SEL_TARGETS_PATH, delimiter=',')
test_data = np.loadtxt(MODEL_ASSESS_DATA_PATH, delimiter=',')
test_targets = np.loadtxt(MODEL_ASSESS_TARGETS_PATH, delimiter=',')

In [4]:
fixed_params = dict(
    n_output_units=3, 
    training_loss_type_value=LossFunction.MSE.value, 
    validation_loss_type_value=LossFunction.MSE.value, 
    evaluation_metric_type_value=Metrics.MEE.value,
    activation_hidden_type_value=ActivationFunction.SIGMOID.value, 
    activation_output_type_value=ActivationFunction.IDENTITY.value, 
    classification=False, 
    early_stopping=True, 
    fast_stopping=False,
    patience=20, 
    tolerance=0.1,
    epochs=500,
    verbose=False,
    hidden_layer_sizes=[64, 32],
    learning_rate=0.0005,
    mom_alpha=0,
    reg_lambda=0,
    batch_size=1,
    nesterov=True,
    linear_decay=True,
)

grid_params = dict(
    tao=[200, 500]
)

results = grid_search(
    k_folds=3, 
    data=train_data, 
    target=train_targets, 
    metrics=[Metrics.MSE.value, Metrics.MEE.value], 
    fixed_param=fixed_params, 
    grid_param=grid_params, 
    file_name_results="test", 
    verbose=True,
    plot=True,
    log_scale=True
)

Grid over n_configurations: 2


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:29<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
fixed_params = dict(
    n_output_units=3, 
    training_loss_type_value=LossFunction.MSE.value, 
    validation_loss_type_value=LossFunction.MSE.value, 
    evaluation_metric_type_value=Metrics.MEE.value,
    activation_hidden_type_value=ActivationFunction.SIGMOID.value, 
    activation_output_type_value=ActivationFunction.IDENTITY.value, 
    classification=False, 
    early_stopping=True, 
    fast_stopping=False,
    patience=20, 
    tolerance=0.1,
    epochs=500,
    verbose=False
)

grid_params = dict(
    hidden_layer_sizes=[[64, 32], [128, 64], [128, 128], [64, 64, 32]],
    learning_rate=[0.0005, 0.005, 0.05, 0.1],
    mom_alpha=[0, 0.6, 0.7, 0.9],
    reg_lambda=[0, 0.0001, 0.00001],
    batch_size=[1, 64, 128],
    nesterov=[True, False],
    linear_decay=[True, False],
    tao=[200, 500]
)

results = grid_search(
    k_folds=3, 
    data=train_data, 
    target=train_targets, 
    metrics=[Metrics.MSE.value, Metrics.MEE.value], 
    fixed_param=fixed_params, 
    grid_param=grid_params, 
    file_name_results="test", 
    verbose=True,
    plot=True,
    log_scale=True
)